这节课主要是讲咋用GPU，条件所限，这里只有一些笔记，代码去colab实现

In [1]:
!nvidia-smi

/bin/bash: nvidia-smi: command not found


In [4]:
import torch
from torch import nn

# 第一个就是访问CPU，第二个访问gpu（不过torch默认把gpu叫cuda......），前面是访问默认的第0块gpu，这里访问第一块cuda:1
torch.device('cpu'), torch.device('cuda'), torch.device('cuda:1')

(device(type='cpu'), device(type='cuda'), device(type='cuda', index=1))

In [5]:
torch.cuda.device_count()
# 没有gpu

0

In [7]:
def try_gpu(i=0):  #@save
    """如果存在 则返回gpu(i) 否则返回cpu()"""
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

def try_all_gpus():  #@save
    """返回所有可用的GPU 如果没有GPU 则返回[cpu(),]"""
    devices = [torch.device(f'cuda:{i}')
             for i in range(torch.cuda.device_count())]
    return devices if devices else [torch.device('cpu')]

In [9]:
try_gpu(), try_gpu(10), try_all_gpus()
# 可以发现我们返回的全部都是cpu

(device(type='cpu'), device(type='cpu'), [device(type='cpu')])

接下来，我们在创建张量的时候，就可以扔在GPU上面了

In [11]:
x = torch.tensor([1, 2, 3])
x.device # 默认是cpu上

device(type='cpu')

In [15]:
X = torch.ones(2, 3, device=try_gpu())
X # 虽然这样还是在cpu上......

tensor([[1., 1., 1.],
        [1., 1., 1.]])

In [16]:
Y = torch.rand(2, 3, device=try_gpu(1))
Y

tensor([[0.7061, 0.0874, 0.1612],
        [0.7505, 0.2920, 0.9288]])

In [19]:
# 当我们进行计算的时候，不同设备（CPU/GPU）的变量是不能一起计算的，比如a=b+c，bc在cuda0上，那就a也会在cuda0上
# 实际上跨设备运行原理上是OK的，但是跨设备特别垃圾，特别是GPU和CPU之间跨设备运算
# 所以这其实是一个约定俗成的办法，我直接不让你算

# Z = X.cuda(1)
# print(X)
# print(Z)

# Y + Z

# Z.cuda(1) is Z

神经网络实际上就是一个大大大函数，那神经网络同样是可以扔在GPU上运算，通过to函数，整体上和上面的张量操作是相同的

In [23]:
net = nn.Sequential(nn.Linear(3, 1))
net = net.to(device=try_gpu())
# 上面这个函数实际上是，把所有的参数/权重放到GPU上
# 所以实际计算的时候，我们也需要记得把传入的样本数据X放到GPU上，不然你实际上也是进行了跨设备运算

In [24]:
net(X)

tensor([[0.4531],
        [0.4531]], grad_fn=<AddmmBackward0>)

In [25]:
net[0].weight.data.device

device(type='cpu')